<a href="https://colab.research.google.com/github/Mancydeep/Assessment-2-weatherwise-/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyinputplus matplotlib requests

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import os
import matplotlib.pyplot as plt
import pyinputplus as pyip

# Add any other setup code here

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Obtain forecast details for a selected city and duration via wttr.in, limiting the days as recommended.
    """
    endpoint = f"https://wttr.in/{location}?format=j1"
    try:
        result = requests.get(endpoint)
        result.raise_for_status()
        data = result.json()
        data['weather'] = data['weather'][:forecast_days]
        return data
    except Exception as error:
        print(f"Unable to retrieve forecast: {error}")
        return None


## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here

def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Display max & min temperatures as stacked horizontal bars with colors.
    """
    dates = []
    max_temps = []
    min_temps = []

    for day in weather_data['weather']:
        dates.append(day['date'])
        max_temps.append(float(day['maxtempC']))
        min_temps.append(float(day['mintempC']))

    plt.figure(figsize=(8, 5))
    y_pos = range(len(dates))

    # Plot min temp as bars, then add max-min stacked over it
    plt.barh(y_pos, min_temps, color='#82CFFD', label='Min Temp (°C)')
    max_min_diff = [max_temps[i] - min_temps[i] for i in range(len(max_temps))]
    plt.barh(y_pos, max_min_diff, left=min_temps, color='#FF6F61', label='Max Temp range')

    plt.yticks(y_pos, dates)
    plt.xlabel("Temperature (°C)")
    plt.title("Temperature Range per Day")
    plt.legend()
    plt.grid(True, axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:
def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Show precipitation totals as dot plot with sizes proportional to rainfall.
    """
    dates = []
    totals = []

    for day in weather_data['weather']:
        dates.append(day['date'])
        total_rain = sum(float(hour['precipMM']) for hour in day['hourly'])
        totals.append(total_rain)

    plt.figure(figsize=(8, 5))
    x_pos = range(len(dates))

    # Variable dot sizes scaled for visibility
    sizes = [t * 100 for t in totals]

    plt.scatter(x_pos, totals, s=sizes, c='#007acc', alpha=0.7, edgecolors='w', linewidth=1.5)
    plt.xticks(x_pos, dates)
    plt.xlabel("Date")
    plt.ylabel("Total Precipitation (mm)")
    plt.title("Rainfall Dot Plot")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:
def create_humidity_visualisation(weather_data, output_type='display'):
    """
    Average humidity per day shown as a smooth gradient filled area chart.
    """
    dates = []
    avg_humidity = []

    for day in weather_data['weather']:
        dates.append(day['date'])
        humid_vals = [float(hour['humidity']) for hour in day['hourly']]
        avg = sum(humid_vals) / len(humid_vals)
        avg_humidity.append(avg)

    plt.figure(figsize=(8, 5))
    plt.plot(dates, avg_humidity, color='#388E3C', lw=2)

    # Color fill with gradient alpha
    for i in range(len(dates)-1):
        plt.fill_between(dates[i:i+2], avg_humidity[i:i+2], color='#A5D6A7', alpha=0.4*(i+1)/len(dates))

    plt.title("Average Daily Humidity")
    plt.xlabel("Date")
    plt.ylabel("Humidity (%)")
    plt.grid(alpha=0.4, linestyle='--')
    plt.tight_layout()
    plt.show()


## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Extracts city, feature, and day from a user's weather query.
    """
    question = question.lower()
    features = ['temperature', 'precipitation']
    time_markers = ['today', 'tomorrow', 'day after tomorrow']
    city = None
    focus = None
    day_ref = 'today'

    for f in features:
        if f in question:
            focus = f
            break
    for t in time_markers:
        if t in question:
            day_ref = t
            break

    parts = question.split()
    if 'in' in parts:
        idx = parts.index('in')
        if idx + 1 < len(parts):
            city = parts[idx + 1]
    else:
        city = 'Perth'
    return {'city': city, 'element': focus, 'period': day_ref}


## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here
def weather_menu():
    """
    Minimal menu interface for weather forecast focusing on temperature and precipitation.
    """
    print("=== Weather Hub ===")
    while True:
        print("\nSelect option:")
        print("1 - Get Overall Weather Summary")
        print("2 - Ask a Weather Question")
        print("3 - Display Temperature Chart")
        print("4 - Display Precipitation Chart")
        print("5 - Exit")

        choice = pyip.inputChoice(['1','2','3','4','5'], prompt="Choose (1-5): ")
        if choice == '5':
            print("Closing Weather Hub. Goodbye!")
            break

        city = pyip.inputStr(prompt="Enter city name: ")
        data = get_weather_data(city, 3)
        if not data:
            print("Error retrieving data. Please try again.")
            continue

        if choice == '1':
            print(generate_weather_response({'city': city, 'element': None, 'period': 'today'}, data))
        elif choice == '2':
            query = pyip


## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a clear response based on temperature or precipitation forecast.
    """
    city = parsed_question['city']
    element = parsed_question['element']
    period = parsed_question['period']
    day_index = 0
    if period == 'tomorrow':
        day_index = 1
    elif period == 'day after tomorrow':
        day_index = 2

    try:
        day = weather_data['weather'][day_index]
        if element in ['temperature', None]:
            high = day['maxtempC']
            low = day['mintempC']
            return f"In {city.title()} on {day['date']}, temperatures will range between {low}°C and {high}°C."
        elif element in ['precipitation', 'rain']:
            total_rain = sum(float(h['precipMM']) for h in day['hourly'])
            return f"In {city.title()} on {day['date']}, total precipitation is expected to be {total_rain:.1f} mm."
        else:
            return "Sorry, only temperature and precipitation data are supported."
    except Exception:
        return "Sorry, weather data is not available for the requested time."


## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.